# Import, Download, & Variable Statements

In [1]:
# Import & download statements
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Variables
extract_path = '/content/drive/MyDrive/RtGender/'
load_path = '/content/drive/MyDrive/RtGender/trns/'

# Load and Clean Source Data

## Define Functions

In [3]:
# specify a file or list of files to load and process
def extract_dfs(filenames):
  if type(filenames) == str: filenames = [filenames]
  df_dict = {}
  # iterate and load dict with dfs
  for file in filenames:
    file_temp = file[:-4] if file[-4:0] == '.csv' else file
    df_temp = pd.read_csv(extract_path + file_temp + '.csv').reset_index()
    df_temp['sourceID'] = df_temp['source'] + df_temp['index'].astype(str)
    df_dict[file] = df_temp

  return df_dict

In [4]:
# split dfs and save to drive
def load_dfs(df_dict):
  for key in df_dict:
    df_temp = df_dict[key]
    X = df_temp.loc[:, df_temp.columns != 'op_gender']
    y = df_temp['op_gender']
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size = 0.2, random_state = 42)
    X_train.to_csv(load_path + key + '_X_train.csv')
    X_test.to_csv(load_path + key + '_X_test.csv')
    y_train.to_csv(load_path + key + '_y_train.csv')
    y_test.to_csv(load_path + key + '_y_test.csv')
  return "Completed!"

## Load Data

In [5]:
raw_file = 'annotations'
df_dict = extract_dfs(raw_file)
load_dfs(df_dict)

'Completed!'